In [6]:
!pip install tensorflow_io==0.23.1 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 25.9 MB 1.4 MB/s 


In [1]:
import tensorflow as tf 
import tensorflow_io as tfio
import matplotlib.pyplot as plt 
import os 

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!cd '/content/drive/MyDrive/Data'

In [ ]:
!unzip '/content/drive/MyDrive/archive(1).zip'

In [3]:
def load_wav_16k_mono(file_name):
  file_contents=tf.io.read_file(file_name)
  wav,sample_rate=tf.audio.decode_wav(file_contents,desired_channels=1)
  wav=tf.squeeze(wav,axis=-1)
  sample_rate=tf.cast(sample_rate,dtype=tf.int64)
  wav=tfio.audio.resample(wav,rate_in=sample_rate,rate_out=16000)
  return wav 

In [2]:
yes=os.path.join('/content/data','Parsed_Capuchinbird_Clips','XC114131-0.wav')
no=os.path.join('/content/data','Parsed_Not_Capuchinbird_Clips','Crickets-chirping-at-night-3.wav')

In [12]:
Neg=os.path.join('/content/data','Parsed_Not_Capuchinbird_Clips')
Pos=os.path.join('/content/data','Parsed_Capuchinbird_Clips')

In [18]:
pos=tf.data.Dataset.list_files(Pos+'/*.wav')
neg=tf.data.Dataset.list_files(Neg+'/*.wav')

In [20]:
positives = tf.data.Dataset.zip((pos, tf.data.Dataset.from_tensor_slices(tf.ones(len(pos)))))
negatives = tf.data.Dataset.zip((neg, tf.data.Dataset.from_tensor_slices(tf.zeros(len(neg)))))
data = positives.concatenate(negatives)

In [24]:
length=[]
for File in os.listdir(os.path.join('/content/data','Parsed_Capuchinbird_Clips')):
  tensor_wav=load_wav_16k_mono(os.path.join('/content/data','Parsed_Capuchinbird_Clips',File))
  length.append(len(tensor_wav))

In [26]:
maxLen=tf.math.reduce_max(length)
meanLen=tf.math.reduce_mean(length)
minLen=tf.math.reduce_min(length)